In [1]:
# Load Packages
import pandas as pd
import numpy as np

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

import keras.models

import model_prep

datapath = "../data"

2023-06-16 09:53:40.391147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
step_back = 6

<h2 style="color:#27469c">1. Preprocess ESB Tower 2 data</h2>

In [3]:
# load data
df = pd.read_csv(f'{datapath}/esb/esb_tower_2_preprocessed.csv', index_col='time')
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(df, season='summer', season_col_name='ESB_Tower_2 season')

# save a boolean series that specifies whether the cooling tower is on
on_condition = df['ESB_Tower_2 fanStatus']

# select features and targets and create final dataframe that includes only relevant features and targets
features = ['ESB_Tower_2 enteringWaterTemp', 'ESB_Tower_2 outdoorAirHumidity', 'ESB_Tower_2 outdoorAirDryBulb', 'ESB_Tower_2 outdoorAirWetBulb', 'Cell_2aFan vfdPower', 'Cell_2bFan vfdPower']
target = 'ESB_Tower_2 leavingWaterTemp'
df = df[features].join(df[target], on=df.index)

# normalize data
scaler = model_prep.NormalizationHandler()
df = scaler.normalize(dtframe=df, target_col=target)

# prepare dataframe for lstm by adding timesteps
lstm_df = model_prep.create_timesteps(df, n_in=step_back, n_out=1, target_name=target)

# remove cases where spring data would leak into summer data (i.e. intial timesteps)
lstm_df = model_prep.remove_irrelevant_data(lstm_df, on_condition, step_back)

# save
lstm_df.to_csv(f"{datapath}/esb/esb2_summer_timestepped.csv")

There are 25784 rows of data for the summer season.
Mininimum ESB_Tower_2 leavingWaterTemp = 0.0
Maximum ESB_Tower_2 leavingWaterTemp = 91.15867614746094
Number of samples in summer data before removing off times: 25784
 Number of samples in summer data after removing off times: 11260


<h2 style="color:#27469c">2. Convert ESB Tower 2 data into a model-compatible shape</h2>

In [4]:
tss = TimeSeriesSplit(n_splits = 3)
X = lstm_df.drop(f'{target}(t)', axis=1) # drop target column
y = lstm_df[f'{target}(t)'] # only have target column

vec_X_test = model_prep.df_to_3d(lstm_dtframe=X, num_columns=len(features)+1, step_back=step_back)

vec_y_test = y.values

print(vec_X_test.shape, vec_y_test.shape)

(11260, 6, 7) (11260,)


<h2 style="color:#27469c">3. Load model and predict</h2>

In [5]:
model = keras.models.load_model('../models_saved/esb1_summer_lstm')

2023-06-16 09:53:44.019385: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
yhat = model.predict(vec_X_test)

352/352 [==============================] - 1s 1ms/step


In [8]:
results_df = pd.DataFrame({'actual': vec_y_test.reshape((vec_y_test.shape[0])), 'predicted': yhat.reshape((yhat.shape[0]))}, index=y.index)
results_df = scaler.denormalize_results(results_df)

# Create a new DataFrame with the desired 5-minute interval index
new_index = pd.date_range(start=results_df.index.min(), end=results_df.index.max(), freq='5min')
display_df = pd.DataFrame(index=new_index)
# Merge the new DataFrame with the original DataFrame
display_df = display_df.merge(results_df, how='left', left_index=True, right_index=True)

fig = px.line(display_df, x=display_df.index, y=["actual", "predicted"])
fig.update_layout(
    title="ESB Tower 1 to ESB Tower 2 LSTM Model Results",
    xaxis_title="time",
    yaxis_title=target
)
fig.show()
fig.write_html("../plots/esb1_to_esb2_summer_lstm.html")